In [2]:
import pandas as pd
import mysql.connector
from datetime import datetime
import os

# establishing connection
conn = mysql.connector.connect(user = 'root',
                               password = ' ',
                               host = 'localhost',
                               database = 'MyCoffeeShop'
                               )
print(conn)
curr = conn.cursor()

In [4]:
year = input("Please select a year to see the Fantasy books solds that year: ")

query7 = f"""
SELECT B.Title
FROM Books as B
JOIN Skews AS S ON S.Skew = B.ISBN
JOIN Sales_Item AS SI on SI.Item = S.Skew
JOIN Sales AS SAL on SAL.Sale_ID = SI.Sale_ID
WHERE B. Genre = 'Fantasy' AND YEAR(SAL.Sale_Date) = {year};
"""
curr.execute(query7)
print(curr.fetchall())

[('Young Warriors: Stories of Strength', 'Fantasy'), ('The Color of Her Panties (Xanth #15)', 'Fantasy'), ("Exile's Children (Exiles  #1)", 'Fantasy'), ('Dragon Quest VIII: Journey of the Cursed King Official Strategy Guide', 'Fantasy'), ('The Sea of Monsters (Percy Jackson and the Olympians  #2)', 'Fantasy'), ('Wolf-Speaker (Immortals  #2)', 'Fantasy')]


In [6]:
# finding most popular breakfast item (couuld also be one where we let the user pick a category)
query = """
    SELECT 
        M.ItemName,
        SUM(SI.Quantity) AS AMTSOLD
    FROM 
        Sales_Item AS SI
    JOIN 
        Menu AS M ON SI.Item = M.MenuItemSkew
    WHERE 
        M.Category = 'Breakfast'
    GROUP BY 
        M.ItemName
    ORDER BY 
        AMTSOLD DESC
    ;
    """
curr.execute(query)
print(curr.fetchall())

[('Plein Continental Breakfast', Decimal('18')), ('Chorizo Burrito', Decimal('18')), ('Plein Omelet', Decimal('12')), ('House Made Biscuit Sandwich', Decimal('11')), ('Breakfast Croissant Sandwich', Decimal('10')), ('Two Eggs Any Style', Decimal('9')), ('Mascarpone Stuffed Brioche French Toast', Decimal('9')), ('Veggie Burrito', Decimal('9'))]


In [19]:
# finding the difference between sales in 2022 and sales in 2023

query = """
    (SELECT 
        SUM(SKEW.Price * SI.Quantity)
     FROM
        Sales_Item AS SI
     JOIN  
        Skews AS SKEW ON SI.Item = SKEW.Skew
     JOIN 
        Sales AS SALES ON SALES.Sale_ID = SI.Sale_ID
     WHERE
        YEAR(SALES.Sale_Date) = 2023)
    -
    (SELECT 
        SUM(SKEW.Price * SI.Quantity)
     FROM
        Sales_Item AS SI
     JOIN  
        Skews AS SKEW ON SI.Item = SKEW.Skew
     JOIN 
        Sales AS SALES ON SALES.Sale_ID = SI.Sale_ID
     WHERE
        YEAR(SALES.Sale_Date) = 2022);
"""
curr.execute(query)
print(curr.fetchall())

[(Decimal('55.45'),)]


In [28]:
# finding all the customer loyalty members who have come more than 3

query = """
SELECT 
    CL.Name, CL.Loyalty_ID, COUNT(S.Sale_ID) AS TOTVISITS
FROM 
    Customer_Loyalty as CL
JOIN
    Sales AS S ON S.Customer_Loyalty_ID = CL.Loyalty_ID
GROUP BY
    CL.Loyalty_ID
HAVING 
    TOTVISITS > 3
ORDER BY
    TOTVISITS DESC
;
"""

curr.execute(query)
print(curr.fetchall())

[('Jaclyn Decker', 202308, 8), ('Alicia Smith', 202630, 6), ('Deborah Valdez', 202161, 4), ('Sharon Buck', 202298, 4), ('Richard Roach', 202352, 4), ('Kim Brown', 202513, 4), ('Mr. Jeremy Mendoza Jr.', 202602, 4), ('Alex Brown', 202675, 4), ('Dana Love', 202762, 4), ('Bryan Cole', 202825, 4), ('Evan Norris', 202910, 4)]


In [35]:
# finding books that havent been sold

query = """
SELECT 
    B.Title
FROM
    Books as B
WHERE 
    B.ISBN NOT IN (
        SELECT 
            SI.Item
        FROM 
            Sales_Item AS SI
    );
"""
curr.execute(query)
print(curr.fetchall())

[('Executive Orders (Jack Ryan  #8)',), ('Warhost of Vastmark (Wars of Light & Shadow  #3; Arc 2 - The Ships of Merior  #2)',), ('Glittering Images (Starbridge  #1)',), ("Miss Marple Omnibus Vol. 3 (Murder at the Vicarage / Nemesis / Sleeping Murder / At Bertram's Hotel)",), ("Zelda's Cut",), ('The Princess of the Chalet School',), ('Partners in Crime (Tommy and Tuppence #2)',), ('Naked Empire (Sword of Truth  #8)',), ('Tales from the Perilous Realm',), ("The Yiddish Policemen's Union",), ('How to Be Alone',), ('Out of the Silent Planet (The Space Trilogy  #1)',), ('I Can Read With My Eyes Shut!',), ('And to Think That I Saw It on Mulberry Street',), ('Horton Hatches the Egg',), ("If I Don't Write It Nobody Else Will: An Autobiography",), ('The Elements of Style',), ('Philosophical Investigations',), ('The Origins of the Civil Rights Movements: Black Communities Organizing for Change',), ('Diary of a Spider',), ('Going Postal (Discworld  #33)',), ('A Year in the Life of William Shakesp

In [41]:
# finding most popular book genre

query = """
SELECT 
    B.Genre, COUNT(SI.Item) as GENCNT
FROM
    Books as B
JOIN 
    Sales_Item AS SI ON SI.Item = B.ISBN
GROUP BY 
    B.Genre
ORDER BY 
    GENCNT DESC;
"""
curr.execute(query)
print(curr.fetchall())

[('Fiction', 67), ('Classics', 58), ('Fantasy', 43), ('History', 32), ('Nonfiction', 28), ('Mystery', 24), ('Science Fiction', 19), ('Sequential Art,Manga', 15), ('Philosophy', 12), ('Historical,Historical Fiction', 11), ('Childrens', 10), ('Poetry', 10), ('Short Stories', 9), ('Horror', 8), ('Sequential Art,Comics', 7), ('Food and Drink,Cookbooks', 7), ('Womens Fiction,Chick Lit', 6), ('Young Adult', 6), ('Spirituality', 6), ('Thriller', 6), ('Business', 5), ('Science', 5), ('Romance', 5), (None, 4), ('Parenting', 4), ('Politics', 4), ('Music', 4), ('Media Tie In,Star Wars', 4), ('Art', 3), ('Sports,Sports', 3), ('Writing,Essays', 3), ('Christian Fiction', 3), ('Culture,Film', 3), ('Romance,Historical Romance', 2), ('Christian', 2), ('Drama', 2), ('Plays,Theatre', 2), ('Adventure', 2), ('Reference', 2), ('Art,Photography', 2), ('Biography', 2), ('Childrens,Picture Books', 2), ('Plays', 2), ('Language,Writing', 2), ('Fantasy,Urban Fantasy', 2), ('Humor', 2), ('Fantasy,Mythology', 1), (

In [42]:
curr.close()
conn.close()